In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, BitsAndBytesConfig
import torch
from qwen_vl_utils import process_vision_info
import nltk
from rouge_score import rouge_scorer

W0529 12:57:38.618000 8692 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Set False for 8-bit
    bnb_4bit_compute_dtype=torch.float16
)

In [4]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto", quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [27]:
filename = "Ramen"
image_path = f"./Test_Images/{filename}.jpeg"
dish = "Soupy Noodles"

In [28]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

In [29]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [30]:
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
description_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(description_text[0])

The image depicts a bowl of ramen noodles served in a white ceramic bowl. The noodles appear to be thick and are garnished with various toppings, including green onions, seaweed, and what looks like pieces of meat or tofu. There are two hard-boiled eggs in the bowl, one of which is cracked open, revealing the runny yolk inside. Additionally, there are some slices of pickled radish and possibly some other vegetables mixed in.

To the left of the bowl, there is a small bowl containing a dark sauce, likely soy sauce or a similar condiment. To the right of the bowl, there are two


In [44]:
recipe_generator = f"""You are a professional recipe generator.
Dish: [Accurate dish name]
Description of the food : {description_text}

Format: The recipe should be in "3-steps" 

Examples: 
Dish: Mustard Fish
Summary: 1. Clean and slice the fish. Soak mustard seeds for 2 hours, then grind with salt and green chillies to a paste. Mix in grated coconut, yoghurt, sugar, turmeric, and mustard oil to make a marinade.
2. Coat the fish pieces with the mixture, place them in a tiffin box with halved green chillies on top, fasten the lid, and marinate for at least 15 minutes.
3. Steam the sealed tiffin in a covered pan with hot water for 15 minutes. Let it rest for 5 minutes before serving.

Dish: Butter Chicken
Summary: 1. Marinate boneless chicken pieces in yogurt, lemon juice, ginger-garlic paste, chili powder, and garam masala for at least 1 hour. Then grill, bake, or pan-fry until cooked and slightly charred.
2. In a pan, heat butter and sauté ginger, garlic, and tomatoes until soft. Blend this into a smooth puree, then return to the pan and add cream, kasuri methi (dried fenugreek), chili powder, and salt. Simmer until rich and creamy.
3. Add the cooked chicken to the sauce, simmer for 10–15 minutes, adjust seasoning, and serve hot with naan or rice.

Dish: Cheesy Bake
Summary: 1. Boil or steam your choice of veggies (e.g., broccoli, cauliflower, carrots, corn) or cooked pasta/chicken. Lightly season with salt, pepper, and herbs.
2. In a pan, melt butter, add flour, and cook for a minute. Slowly whisk in milk to make a smooth white sauce. Add grated cheese (like cheddar or mozzarella) and stir until melted and creamy.
3. Mix the base with the cheese sauce, pour into a greased baking dish, top with more cheese, and bake at 180°C (350°F) for 20–25 minutes until golden and bubbly. Serve hot!"""

In [45]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": recipe_generator},
        ],
    }
]

In [46]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

In [47]:
generated_ids = model.generate(
    **inputs,
    max_new_tokens=300,
    ).to("cpu")
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
recipe_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
generated_recipe = recipe_text[0]
print(recipe_text[0])

Dish: Ramen Noodles
Summary: 1. Cook the noodles according to package instructions. While the noodles are cooking, prepare the broth by combining chicken stock, soy sauce, sesame oil, and any desired seasonings such as garlic, ginger, and sesame seeds. Bring to a boil over medium-high heat and reduce to a simmer.
2. Once the noodles are cooked, drain them and set aside. In a separate pot, cook the meatballs or tofu cubes in the same broth for a few minutes until they are heated through.
3. To assemble the bowl, place the noodles in the center of a large bowl. Add the meatballs or tofu cubes, garnish with green onions, seaweed, and any additional toppings such as pickled radish or scallions. Place the hard-boiled eggs on top, cracked open if desired. Serve immediately with chopsticks for easy eating.


In [48]:
reference_summary_path = f"./Test_Summaries/{filename}.txt"
with open(reference_summary_path, "r", encoding="utf-8") as ref_file:
    reference_summary = ref_file.read().strip()

In [49]:
scorer = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeL'], 
    use_stemmer=True
)

In [50]:
scores = scorer.score(reference_summary, generated_recipe)

In [51]:
print("ROUGE Scores:")
for metric, score in scores.items():
    print(f"{metric.upper()}:")
    print(f"  Precision: {score.precision:.4f}")
    print(f"  Recall:    {score.recall:.4f}")
    print(f"  F1:        {score.fmeasure:.4f}")
    print()

ROUGE Scores:
ROUGE1:
  Precision: 0.3071
  Recall:    0.6324
  F1:        0.4135

ROUGE2:
  Precision: 0.0791
  Recall:    0.1642
  F1:        0.1068

ROUGEL:
  Precision: 0.1714
  Recall:    0.3529
  F1:        0.2308

